# 홍보글 데이터 만들기 

### 홍보글 데이터 예시 

```
서울책사랑 동호회 신규 회원 모집 안내

안녕하세요, 책을 사랑하는 모든 분들을 환영합니다! 저희 서울책사랑 동호회는 다양한 장르의 책을 함께 읽고, 토론하며, 새로운 인연을 만들어가는 모임입니다. 이번에 신규 회원을 모집하오니 많은 관심과 참여 부탁드립니다. 📚✨

활동 내용: 매달 선정된 도서를 읽고 토론하는 북클럽 모임, 작가 초청 강연, 독서 여행 등 다양한 활동을 진행합니다.

모임 일시: 매월 첫째 주 토요일 오후 2시

장소: 서울시 종로구 종로3가 12-34, 3층 카페북스

가입 방법: 아래 연락처로 성함과 연락처를 보내주시면 가입 절차를 안내해 드리겠습니다.

연락처:
- 이메일: seoulbooklove@gmail.com
- 전화번호: 010-2345-6789
- 카카오톡 ID: booklover_seoul

회비: 월 10,000원 (우리은행 1002-123-456789 홍길동)

책을 통해 삶을 더 풍요롭게 만들고 싶은 분이라면 누구든지 환영합니다. 많은 참여 부탁드립니다! 😊📖

문의사항은 언제든지 연락 주세요. 감사합니다!
```

### 비식별화 데이터 예시

```
서울책사랑 동호회 신규 회원 모집 안내

안녕하세요, 책을 사랑하는 모든 분들을 환영합니다! 저희 서울책사랑 동호회는 다양한 장르의 책을 함께 읽고, 토론하며, 새로운 인연을 만들어가는 모임입니다. 이번에 신규 회원을 모집하오니 많은 관심과 참여 부탁드립니다. 📚✨

활동 내용: 매달 선정된 도서를 읽고 토론하는 북클럽 모임, 작가 초청 강연, 독서 여행 등 다양한 활동을 진행합니다.

모임 일시: 매월 첫째 주 토요일 오후 2시

장소: [LOCATION1], 3층 카페북스

가입 방법: 아래 연락처로 성함과 연락처를 보내주시면 가입 절차를 안내해 드리겠습니다.

연락처:
- 이메일: [EMAIL1]
- 전화번호: [CONTACT1]
- 카카오톡 ID: [KAKAO_ID1]

회비: 월 10,000원 ([ACCOUNT1] [PERSON1])

책을 통해 삶을 더 풍요롭게 만들고 싶은 분이라면 누구든지 환영합니다. 많은 참여 부탁드립니다! 😊📖

문의사항은 언제든지 연락 주세요. 감사합니다!
```

In [1]:
import re
import os

import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from pqdm.processes import pqdm
from datasets import load_dataset

# 환경변수 로드
load_dotenv("./credit-env")
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))

# 데이터셋 로드
ds = load_dataset("daekeun-ml/naver-news-summarization-ko")
ds

/home/admin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [2]:
ds["train"][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [3]:
# config들 설정
## ✅ 카테고리 리스트 설정
categories = [
    "제품_리뷰글",
    "중고나라_게시물",
    "이벤트_안내글",
    "강의_세미나_홍보글",
    "모임_동호회_홍보글"
]
## ✅ 데이터 개수 설정
num_samples = 120  # 한 카테고리당 몇 개 생성할지 설정
N_JOBS = 20

## ✅ Reference 데이터 가져오기
reference_data_list = ds["train"]["title"][:num_samples]


In [4]:
# 데이터 생성
### ✅ 데이터 생성 프롬프트
data_system_prompt = f"""당신은 한국어로 다양한 카테고리의 현실적이고 자연스러운 데이터를 생성하는 어시스턴트입니다."""

data_user_prompt = f"""당신은 가상의 모임 또는 {{category}}을 생성해야 합니다.
모임명, 활동 내용, 모임 일시, 장소, 가입 방법, 연락처(이메일, 전화번호 등) 등을 작성하세요.
각 데이터는 실제로 존재할 법한 자유로운 형태로 작성되며, 반드시 아래의 개인정보 중 최소 3가지 이상을 포함해야 합니다.

- 사람 이름
- 연락처 (전화번호, 이메일, 카카오톡 ID 등)
- 주소
- 계좌번호
- 소셜미디어 ID (트위터, 텔레그램 등)

작성할 때 개인정보(이름, 연락처, 계좌번호, 주소 등)를 구체적으로 포함해야 합니다. 
데이터의 전체 길이는 최소 300자 이상, 최대 1000자 이하로 작성합니다.
제목 등에 [모집], [중고], [서울책사랑 동호회 안내] 등 [와 ]를 절대 사용하지 마세요.
절대 마크다운 형식으로 작성하지 마세요. 
필요하다면 이모티콘으로 예쁘게 꾸며주세요.
아래 reference로부터 아이디어를 얻어 {{category}}에 대해 현실적이고 자연스러운 데이터를 작성하세요.

reference:::
{{reference_data}}
reference:::

자! 시작
"""

### 1️⃣ 데이터 생성 함수
def generate_data(args):
    category, reference_data = args
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": data_system_prompt},
            {"role": "user", "content": data_user_prompt.format(category=category, reference_data=reference_data)}
        ],
        temperature=0.5
    )
    return {
        "data": response.choices[0].message.content, 
        "category": category, 
        "prompt": data_user_prompt.format(
            category=category, 
            reference_data=reference_data)
            }

### ✅ 병렬 데이터 생성
print(f"{', '.join(categories)} 등의 주어진 카테고리에 대해 데이터 생성 중...")
generated_data_list = pqdm(
    [(category, reference_data) for category in categories for reference_data in reference_data_list],
    generate_data,
    n_jobs=N_JOBS
)

### 데이터 생성 결과 예시
generated_data_list[:2]

제품_리뷰글, 중고나라_게시물, 이벤트_안내글, 강의_세미나_홍보글, 모임_동호회_홍보글 등의 주어진 카테고리에 대해 데이터 생성 중...


QUEUEING TASKS | : 100%|██████████| 600/600 [00:00<00:00, 2633.60it/s]
PROCESSING TASKS | : 100%|██████████| 600/600 [03:00<00:00,  3.33it/s]
COLLECTING RESULTS | : 100%|██████████| 600/600 [00:00<00:00, 167895.28it/s]


[{'data': "서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 사랑하고 함께 읽고 토론하는 것을 즐기는 사람들의 모임입니다. 매월 첫째 주 토요일 오후 2시에 서울 종로구 북촌로 35, 3층 카페 '북카페'에서 모임을 가집니다. 이번 달에는 하퍼 리의 '앵무새 죽이기'를 읽고 토론할 예정입니다.\n\n가입 방법은 간단합니다. 카카오톡 ID 'SeoulBookLovers'로 메시지를 보내주시면 됩니다. 첫 모임은 무료로 참석할 수 있으니 부담 없이 오세요! 이후 정기회원으로 가입하실 경우 월 1만원의 회비가 있습니다. 회비는 우리은행 1002-123-456789 (예금주: 김서현)으로 송금하시면 됩니다.\n\n문의 사항이 있으시면 언제든지 이메일 seoulbooklovers@gmail.com 또는 전화 010-2345-6789로 연락 주세요. 책을 사랑하는 여러분과 함께하는 시간을 기대하고 있습니다! 😊📖\n\n참고로, 저희 모임은 다양한 연령대와 직업군의 사람들이 모여 서로의 생각을 나누는 열린 공간입니다. 책을 좋아하는 사람이라면 누구나 환영합니다. 이번 기회에 새로운 친구도 만나고, 깊이 있는 독서 경험을 함께 나누어 보세요!",
  'category': '제품_리뷰글',
  'prompt': '당신은 가상의 모임 또는 제품_리뷰글을 생성해야 합니다.\n모임명, 활동 내용, 모임 일시, 장소, 가입 방법, 연락처(이메일, 전화번호 등) 등을 작성하세요.\n각 데이터는 실제로 존재할 법한 자유로운 형태로 작성되며, 반드시 아래의 개인정보 중 최소 3가지 이상을 포함해야 합니다.\n\n- 사람 이름\n- 연락처 (전화번호, 이메일, 카카오톡 ID 등)\n- 주소\n- 계좌번호\n- 소셜미디어 ID (트위터, 텔레그램 등)\n\n작성할 때 개인정보(이름, 연락처, 계좌번호, 주소 등)를 구체적으로 포함해야 합니다. \n데이터의 전체 길이는 최소 300자 이상, 최대 1000자 이하로 작성합니다.\n제목 등에 [모집], [중

In [5]:
# 글에 비식별화 정량평가에 사용할 []가 있는 데이터는 제거 
original_data_list = generated_data_list
print(f"생성된 데이터 수 : {len(generated_data_list)}")
generated_data_list = [article for article in generated_data_list if "[" not in article["data"]]
print(f"필터링된 데이터 수 : {len(generated_data_list)}")
print(f"필터링된 데이터수 : {len(original_data_list) - len(generated_data_list)}")

생성된 데이터 수 : 600
필터링된 데이터 수 : 600
필터링된 데이터수 : 0


In [6]:
# OutputFormat 클래스는 OpenAI의 "structured data" 응답을 받을 때 사용할 데이터 형식을 정의합니다.
# OpenAI의 structured response_format을 사용할 때 응답을 json 형태로 받기 위해 사용하는 클래스입니다.
# API 호출 시 이 클래스에서 지정한 형태의 데이터를 반환하도록 GPT 모델에게 지시할 수 있습니다.
# 여기서는 평가 점수(score)와 평가 이유(reason)를 받습니다.

class OutputFormat(BaseModel):
    score: int  # 데이터 품질 평가 점수 (1~5)
    reason: str  # 해당 점수를 매긴 이유 (한국어로 설명)

In [7]:
### 2️⃣ 데이터 평가 함수

data_valid_system_prompt = "당신은 프롬프트와 데이터 품질 평가하는 어시스턴트입니다."

data_valid_user_prompt = f"""주어진 프롬프트에 맞게 데이터가 생성되었는지 평가하세요.

# 프롬프트
{{prompt}}


# 데이터
{{data}}

# 데이터 평가 기준:
- 개인정보 포함: 이름, 생년월일, 연락처 등 최소 3가지 이상의 개인정보가 포함되었는가? 개인정보를 많이 가지고 있으면 있을수록 좋습니다.
- 개인정보 포맷: 개인정보는 현실에서 나올 법한 포맷을 갖추고 있는가?
- 현실적 대화: 생성된 대화는 현실에서 나올 법한 자연스러운 문맥을 갖추고 있는가?
- 문장의 길이: 데이터는 최소 300자 이상, 최대 1000자 이하의 길이를 갖추었는가?

# 평가 점수:
- 모든 조건을 완벽히 만족: 5점
- 1개의 평가 조건이 부족함: 4점
- 2개의 평가 조건이 부족함: 3점
- 3개의 평가 조건이 부족함: 2점
- 4개의 평가 조건이 부족함: 1점

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""


def validate_data(args):
    import json_repair
    data, category, prompt = args
    ####################
    ### beta 모델 호출 ###
    ####################
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": data_valid_system_prompt},
            {"role": "user", "content": data_valid_user_prompt.format(prompt=prompt, data=data)}
        ],
        temperature=0.5,
        response_format=OutputFormat
    )
    ################
    ### json 파싱 ###
    ################
    result = json_repair.loads(response.choices[0].message.content)
    
    return {
        "generated_data": data,
        "category": category, 
        "generate_score": result["score"], 
        "generate_reason": result["reason"], 
        }

# ✅ 병렬 데이터 검증
print(f"{', '.join(categories)} 주어진 카테고리에 대해 데이터 검증 중...")
validated_data_list = pqdm(
    [(args["data"], args["category"], args["prompt"]) for args in generated_data_list],
    validate_data,
    n_jobs=N_JOBS
)

# ✅ score 5인 데이터만 선별
high_quality_data = [item for item in validated_data_list if item["generate_score"] == 5]

print("5점 이상의 데이터 개수:", len(high_quality_data))
print("5점 이상의 데이터 예시:")
high_quality_data[:2]

제품_리뷰글, 중고나라_게시물, 이벤트_안내글, 강의_세미나_홍보글, 모임_동호회_홍보글 주어진 카테고리에 대해 데이터 검증 중...


QUEUEING TASKS | : 100%|██████████| 600/600 [00:00<00:00, 2304.39it/s]
PROCESSING TASKS | : 100%|██████████| 600/600 [02:05<00:00,  4.78it/s]
COLLECTING RESULTS | : 100%|██████████| 600/600 [00:00<00:00, 184649.09it/s]

5점 이상의 데이터 개수: 454
5점 이상의 데이터 예시:


[{'generated_data': "서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 사랑하고 함께 읽고 토론하는 것을 즐기는 사람들의 모임입니다. 매월 첫째 주 토요일 오후 2시에 서울 종로구 북촌로 35, 3층 카페 '북카페'에서 모임을 가집니다. 이번 달에는 하퍼 리의 '앵무새 죽이기'를 읽고 토론할 예정입니다.\n\n가입 방법은 간단합니다. 카카오톡 ID 'SeoulBookLovers'로 메시지를 보내주시면 됩니다. 첫 모임은 무료로 참석할 수 있으니 부담 없이 오세요! 이후 정기회원으로 가입하실 경우 월 1만원의 회비가 있습니다. 회비는 우리은행 1002-123-456789 (예금주: 김서현)으로 송금하시면 됩니다.\n\n문의 사항이 있으시면 언제든지 이메일 seoulbooklovers@gmail.com 또는 전화 010-2345-6789로 연락 주세요. 책을 사랑하는 여러분과 함께하는 시간을 기대하고 있습니다! 😊📖\n\n참고로, 저희 모임은 다양한 연령대와 직업군의 사람들이 모여 서로의 생각을 나누는 열린 공간입니다. 책을 좋아하는 사람이라면 누구나 환영합니다. 이번 기회에 새로운 친구도 만나고, 깊이 있는 독서 경험을 함께 나누어 보세요!",
  'category': '제품_리뷰글',
  'generate_score': 5,
  'generate_reason': '데이터는 모든 평가 기준을 완벽히 만족하고 있습니다. 개인정보는 이름(김서현), 연락처(전화번호, 이메일, 카카오톡 ID), 계좌번호 등 5가지가 포함되어 있으며, 각각 현실적인 포맷을 갖추고 있습니다. 생성된 문장은 매우 자연스럽고 현실적인 대화로 구성되어 있으며, 글의 길이도 300자 이상, 1000자 이하로 적절하게 작성되었습니다. 따라서 평가 점수는 5점입니다.'},
 {'generated_data': '서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울책사랑 동호회입니다! 저희 모임은 매월 첫째 주 토요일 오후 

In [8]:
anonymized_system_prompt = f"""당신은 한국어로 데이터 내에 포함된 모든 개인정보를 placeholder로 비식별화하는 작업을 수행하는 어시스턴트입니다."""

anonymized_user_prompt = f"""입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 처리합니다. 동일 인물의 개인정보는 같은 번호를 사용하여 일관성을 유지해야 합니다.
입력 데이터를 바탕으로, 아래 placeholder를 사용하여 모든 개인정보를 비식별화하세요.

intput:::
{{text}}
intput:::

| 개인정보 종류 | placeholder 예시       |
|---------------|------------------------|
| 이름          | `[PERSON1]`, `[PERSON2]` 등 |
| 생년월일      | `[DATEOFBIRTH1]`, `[DATEOFBIRTH2]` 등 |
| 연락처        | `[CONTACT1]`, `[CONTACT2]` 등 |
| 주소          | `[ADDRESS1]`, `[ADDRESS2]` 등 |
| 계좌번호      | `[ACCOUNT1]`, `[ACCOUNT2]` 등 |
| 이메일        | `[EMAIL1]`, `[EMAIL2]` 등 |
| 장소 및 지역명 | `[LOCATION1]`, `[LOCATION2]` 등 |
| 카카오톡 ID   | `[KAKAO_ID1]`, `[KAKAO_ID2]` 등 |
| 트위터 ID     | `[TWITTER_ID1]`, `[TWITTER_ID2]` 등 |
| 텔레그램 ID   | `[TELEGRAM_ID1]`, `[TELEGRAM_ID2]` 등 |


자! 시작
"""

### 3️⃣ 비식별화 함수
def anonymize_data(args):
    generated_data, category, generate_score, generate_reason = args
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": anonymized_system_prompt},
            {"role": "user", "content": anonymized_user_prompt.format(text=generated_data)}
        ],
        temperature=0.1
    )
    return {
        "generated_data": generated_data, 
        "category": category, 
        "generate_score": generate_score, 
        "generate_reason": generate_reason,
        "anonymized_data": response.choices[0].message.content, 
        "anonymized_prompt": anonymized_user_prompt.format(text=generated_data)
        }

# ✅ 병렬 비식별화 처리
anonymized_data_list = pqdm(
    [(
        item["generated_data"], 
        item["category"], 
        item["generate_score"], 
        item["generate_reason"]
        ) for item in high_quality_data],
    anonymize_data,
    n_jobs=N_JOBS
)


QUEUEING TASKS | : 100%|██████████| 454/454 [00:00<00:00, 1763.28it/s]
PROCESSING TASKS | : 100%|██████████| 454/454 [02:19<00:00,  3.25it/s]
COLLECTING RESULTS | : 100%|██████████| 454/454 [00:00<00:00, 161305.72it/s]


In [9]:
anonymized_data_list[:1]

[{'generated_data': "서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 사랑하고 함께 읽고 토론하는 것을 즐기는 사람들의 모임입니다. 매월 첫째 주 토요일 오후 2시에 서울 종로구 북촌로 35, 3층 카페 '북카페'에서 모임을 가집니다. 이번 달에는 하퍼 리의 '앵무새 죽이기'를 읽고 토론할 예정입니다.\n\n가입 방법은 간단합니다. 카카오톡 ID 'SeoulBookLovers'로 메시지를 보내주시면 됩니다. 첫 모임은 무료로 참석할 수 있으니 부담 없이 오세요! 이후 정기회원으로 가입하실 경우 월 1만원의 회비가 있습니다. 회비는 우리은행 1002-123-456789 (예금주: 김서현)으로 송금하시면 됩니다.\n\n문의 사항이 있으시면 언제든지 이메일 seoulbooklovers@gmail.com 또는 전화 010-2345-6789로 연락 주세요. 책을 사랑하는 여러분과 함께하는 시간을 기대하고 있습니다! 😊📖\n\n참고로, 저희 모임은 다양한 연령대와 직업군의 사람들이 모여 서로의 생각을 나누는 열린 공간입니다. 책을 좋아하는 사람이라면 누구나 환영합니다. 이번 기회에 새로운 친구도 만나고, 깊이 있는 독서 경험을 함께 나누어 보세요!",
  'category': '제품_리뷰글',
  'generate_score': 5,
  'generate_reason': '데이터는 모든 평가 기준을 완벽히 만족하고 있습니다. 개인정보는 이름(김서현), 연락처(전화번호, 이메일, 카카오톡 ID), 계좌번호 등 5가지가 포함되어 있으며, 각각 현실적인 포맷을 갖추고 있습니다. 생성된 문장은 매우 자연스럽고 현실적인 대화로 구성되어 있으며, 글의 길이도 300자 이상, 1000자 이하로 적절하게 작성되었습니다. 따라서 평가 점수는 5점입니다.',
  'anonymized_data': "서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 사랑하고 함께 읽고 토론하는 것을 즐기는 사람들의 모임입니다.

In [10]:
anonymized_valid_system_prompt = "당신은 비식별화 데이터 품질 평가하는 어시스턴트입니다."

anonymized_valid_user_prompt = f"""당신에게 비식별화 프롬프트와 비식별화 데이터의 품질을 평가하세요. 

# 비식별화 프롬프트
{{prompt}}


# 비식별화 데이터
{{data}}

origin_data
# 검증 기준:  
1. 모든 개인정보가 비식별화되었는가?  
   - 이름, 생년월일, 연락처, 주소, 계좌번호 등 모든 개인정보가 placeholder로 대체되었는가?  
2. 동일한 개인정보는 일관된 placeholder를 사용하는가?  
   - 같은 인물, 같은 이메일, 같은 연락처 등이 일관된 ID로 처리되었는가?  
3. 비식별화 후 문장이 자연스러운가?  
   - 문맥이 손상되지 않고, 대화 흐름이 유지되는가?  

# 평가 점수:  
- 5점: 모든 기준을 완벽하게 충족 (개인정보가 누락 없이 비식별화되었고, 일관성이 유지되며 문장이 자연스러움)  
- 4점: 거의 완벽하지만 일부 개인정보가 누락되었거나 일관성이 살짝 부족함  
- 3점: 여러 개인정보가 비식별화되지 않았거나, 동일 개인정보의 placeholder가 일관되지 않음  
- 2점: 대부분의 개인정보가 비식별화되지 않았으며, placeholder 사용이 올바르지 않음  
- 1점: 개인정보가 거의 그대로 남아 있거나, 잘못된 방식으로 변환됨

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""

### 4️⃣ 비식별화 검증 함수
def validate_anonymized_data(args):
    ( 
        generated_data, 
        category, 
        generate_score, 
        generate_reason, 
        anonymized_data,
        anonymized_prompt  ) = args
    ####################
    ### beta 모델 호출 ###
    ####################
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": anonymized_valid_system_prompt},
            {"role": "user", "content": anonymized_valid_user_prompt.format(
                prompt=anonymized_prompt, 
                data=anonymized_data
                )
            }
        ],
        temperature=0.4,
        response_format=OutputFormat
    )
    result = response.choices[0].message.content
    ################
    ### json 파싱 ###
    ################
    result = json_repair.loads(result)
    return {
                "origin_data": generated_data, 
                "category": category, 
                "generate_score": generate_score,
                "generate_reason": generate_reason,
                "anonymized_data": anonymized_data,
                "anonymized_prompt": anonymized_prompt,
                "validate_score": result["score"], 
                "validate_reason": result["reason"]
            }

# ✅ 병렬 비식별화 검증
validated_anonymized_data_list = pqdm(
    [(
        item["generated_data"], 
        item["category"], 
        item["generate_score"],
        item["generate_reason"], 
        item["anonymized_data"], 
        item["anonymized_prompt"] ) for item in anonymized_data_list],
    validate_anonymized_data,
    n_jobs=N_JOBS
)

QUEUEING TASKS | : 100%|██████████| 454/454 [00:00<00:00, 1769.13it/s]
PROCESSING TASKS | : 100%|██████████| 454/454 [01:10<00:00,  6.45it/s]
COLLECTING RESULTS | : 100%|██████████| 454/454 [00:00<00:00, 187275.18it/s]


In [11]:
validated_anonymized_data_list[1]

{'origin_data': '서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울책사랑 동호회입니다! 저희 모임은 매월 첫째 주 토요일 오후 2시에 서울 종로구 인사동길 23번지, 북카페 \'책과 커피\'에서 진행됩니다. 📚☕\n\n이번 달의 주제는 "현대 사회와 인간관계"로, 관련 도서를 읽고 자유롭게 의견을 나눌 예정입니다. 참여를 원하시는 분들은 아래의 연락처로 가입 신청을 해주세요. \n\n가입 방법은 간단합니다. 김민수 회장에게 이메일(booklover@naver.com)이나 전화(010-2345-6789)로 연락 주시면 됩니다. 또한, 카카오톡 ID \'seoulbookclub\'으로도 문의 가능합니다. 😊\n\n저희 모임은 다양한 배경을 가진 사람들이 모여 서로의 경험과 생각을 공유하는 자리입니다. 독서를 통해 서로의 생각을 넓히고, 새로운 친구들을 만나고 싶으신 분들을 환영합니다. \n\n가입비는 없으며, 모임 당일 커피와 간단한 다과가 제공됩니다. 책을 사랑하는 모든 분들을 기다리고 있습니다. 함께 책을 읽고, 나누는 즐거움을 느껴보세요! 📖☕',
 'category': '제품_리뷰글',
 'generate_score': 5,
 'generate_reason': '주어진 데이터는 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 데이터에는 사람 이름(김민수), 연락처(전화번호: 010-2345-6789, 이메일: booklover@naver.com), 소셜미디어 ID(카카오톡 ID: seoulbookclub) 등 3가지 이상의 개인정보가 포함되어 있습니다.\n\n2. 개인정보 포맷: 개인정보는 현실에서 나올 법한 포맷을 갖추고 있습니다. 전화번호와 이메일, 카카오톡 ID 모두 일반적으로 사용되는 형식을 따르고 있습니다.\n\n3. 현실적 대화: 모임 안내문은 자연스럽고 현실적인 문맥을 가지고 있으며, 독서 모임에 대한 관심을 끌 수 있는 내용으로 구성되어 있습니다.\n\n4. 문장의 길이: 데이터의 전

In [12]:
print("검증데이터 개수:", len(validated_anonymized_data_list))
# ✅ 최종 데이터 선별 (비식별화 평가 5점만)
final_data = [item for item in validated_anonymized_data_list if item["validate_score"] == 5]
print("최종데이터 개수:", len(final_data))

검증데이터 개수: 454
최종데이터 개수: 360


In [13]:
import pandas as pd 
df = pd.DataFrame(final_data)
print("최종 데이터 개수:", df.shape)
df = df[(df["generate_score"] == 5) & (df["validate_score"] == 5)]
print("모두 5점 이상 데이터 개수:", df.shape)
df.head(2)

최종 데이터 개수: (360, 8)
모두 5점 이상 데이터 개수: (360, 8)


,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason
0,"서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 ...",제품_리뷰글,5,"데이터는 모든 평가 기준을 완벽히 만족하고 있습니다. 개인정보는 이름(김서현), 연...","서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 ...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"1. 모든 개인정보가 적절히 비식별화되었습니다. 이름, 연락처, 주소, 계좌번호, ..."
1,"서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울...",제품_리뷰글,5,주어진 데이터는 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 데이터...,"서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"1. 모든 개인정보가 비식별화되었음: 이름, 연락처, 이메일, 카카오톡 ID, 장소..."


In [14]:
def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping


### extract_placeholder_mapping 함수의 결과 

- 입력 : (홍보글 데이터, 비식별화 데이터)
- 출력 
    * 서울 -> [LOCATION1]
    * 서울 강남구 테헤란로 25, 3층 -> [ADDRESS1]
    * 강남역 -> [LOCATION2]
    * seoulfoodlovers@gmail.com -> [EMAIL1]
    * 김지훈 -> [PERSON1]
    * 010-1234-5678 -> [CONTACT1]

In [15]:
# mapping 예시 추출
mapping = extract_placeholder_mapping(
    df["origin_data"].iloc[0], 
    df["anonymized_data"].iloc[0],
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", 
        "ACCOUNT", "DATEOFBIRTH", "EMAIL", 
        "LOCATION", "KAKO_ID", "TWITTER_ID", "TELEGRAM_ID"
        )
)

for orig_val, ph in mapping.items():
    print(f"{orig_val} -> {ph}")


서울 -> [LOCATION1]
종로구 북촌로 35, 3층 카페 '북카페' -> [ADDRESS1]
우리은행 1002-123-456789 -> [ACCOUNT1]
김서현 -> [PERSON1]
seoulbooklovers@gmail.com -> [EMAIL1]
010-2345-6789 -> [CONTACT1]


In [16]:
df["mapping"] = df.apply(lambda x: extract_placeholder_mapping(
    x["origin_data"], 
    x["anonymized_data"], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TWITTER_ID", "TELEGRAM_ID")), 
    axis=1)

df.head(2)

,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason,mapping
0,"서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 ...",제품_리뷰글,5,"데이터는 모든 평가 기준을 완벽히 만족하고 있습니다. 개인정보는 이름(김서현), 연...","서울책사랑 동호회 안내\n\n안녕하세요, 서울책사랑 동호회입니다! 📚 저희는 책을 ...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"1. 모든 개인정보가 적절히 비식별화되었습니다. 이름, 연락처, 주소, 계좌번호, ...","{'서울': '[LOCATION1]', '종로구 북촌로 35, 3층 카페 '북카페'..."
1,"서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울...",제품_리뷰글,5,주어진 데이터는 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 데이터...,"서울책사랑 동호회 안내\n\n안녕하세요, 독서와 토론을 사랑하는 여러분을 위한 서울...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"1. 모든 개인정보가 비식별화되었음: 이름, 연락처, 이메일, 카카오톡 ID, 장소...","{'서울 종로구 인사동길 23번지': '[LOCATION1]', '책과 커피': '..."


In [17]:
df[df["mapping"] == {}]

,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason,mapping
13,"서울책사랑 동호회 안내\n\n안녕하세요, 책을 사랑하는 모든 분들을 위한 '서울책사...",제품_리뷰글,5,데이터는 다음과 같은 이유로 모든 평가 조건을 완벽히 만족합니다:\n\n1. **개...,"```\n[LOCATION1] 동호회 안내\n\n안녕하세요, 책을 사랑하는 모든 분...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"1. 모든 개인정보가 비식별화되었는가? \n - 이름, 연락처, 주소, 이메일,...",{}
128,"서울책사랑 동호회 안내\n\n안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에...",중고나라_게시물,5,데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 이름(김...,입력 데이터를 바탕으로 개인정보를 비식별화한 결과는 다음과 같습니다:\n\n```\...,입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"비식별화된 데이터는 모든 개인정보를 정확하게 placeholder로 대체하였으며, ...",{}


In [24]:
print(df["origin_data"].iloc[200])

서울책사랑 동호회 신규 회원 모집 안내

안녕하세요, 책을 사랑하는 모든 분들을 환영합니다! 저희 서울책사랑 동호회는 다양한 장르의 책을 함께 읽고, 토론하며, 새로운 인연을 만들어가는 모임입니다. 이번에 신규 회원을 모집하오니 많은 관심과 참여 부탁드립니다. 📚✨

활동 내용: 매달 선정된 도서를 읽고 토론하는 북클럽 모임, 작가 초청 강연, 독서 여행 등 다양한 활동을 진행합니다.

모임 일시: 매월 첫째 주 토요일 오후 2시

장소: 서울시 종로구 종로3가 12-34, 3층 카페북스

가입 방법: 아래 연락처로 성함과 연락처를 보내주시면 가입 절차를 안내해 드리겠습니다.

연락처:
- 이메일: seoulbooklove@gmail.com
- 전화번호: 010-2345-6789
- 카카오톡 ID: booklover_seoul

회비: 월 10,000원 (우리은행 1002-123-456789 홍길동)

책을 통해 삶을 더 풍요롭게 만들고 싶은 분이라면 누구든지 환영합니다. 많은 참여 부탁드립니다! 😊📖

문의사항은 언제든지 연락 주세요. 감사합니다!


In [25]:
print(df["anonymized_data"].iloc[200])

서울책사랑 동호회 신규 회원 모집 안내

안녕하세요, 책을 사랑하는 모든 분들을 환영합니다! 저희 서울책사랑 동호회는 다양한 장르의 책을 함께 읽고, 토론하며, 새로운 인연을 만들어가는 모임입니다. 이번에 신규 회원을 모집하오니 많은 관심과 참여 부탁드립니다. 📚✨

활동 내용: 매달 선정된 도서를 읽고 토론하는 북클럽 모임, 작가 초청 강연, 독서 여행 등 다양한 활동을 진행합니다.

모임 일시: 매월 첫째 주 토요일 오후 2시

장소: [LOCATION1], 3층 카페북스

가입 방법: 아래 연락처로 성함과 연락처를 보내주시면 가입 절차를 안내해 드리겠습니다.

연락처:
- 이메일: [EMAIL1]
- 전화번호: [CONTACT1]
- 카카오톡 ID: [KAKAO_ID1]

회비: 월 10,000원 ([ACCOUNT1] [PERSON1])

책을 통해 삶을 더 풍요롭게 만들고 싶은 분이라면 누구든지 환영합니다. 많은 참여 부탁드립니다! 😊📖

문의사항은 언제든지 연락 주세요. 감사합니다!


In [18]:
final_df = df[df["mapping"] != {}]
final_df.shape

(358, 9)

In [19]:
import time
for category in categories:
    sub_df = final_df[final_df["category"] == category]
    with open(
        f"./data/{category}_final_{time.strftime('%Y%m%d_%H%M%S')}.csv", 
        "w", 
        encoding="utf-8") as f:
        f.write(sub_df.to_csv(index=False))

print("✅ 모든 카테고리 데이터 처리 완료 🚀")

✅ 모든 카테고리 데이터 처리 완료 🚀
